In [6]:
import os
import time

import h5py
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from cv2 import imwrite
from keras import Input, Model
from keras.callbacks import TensorBoard
from keras.layers import Convolution2D, LeakyReLU, BatchNormalization, UpSampling2D, Dropout, Activation, Flatten, \
    Dense, Lambda, Reshape, concatenate
from keras.optimizers import Adam
import tensorflow as tf


Using TensorFlow backend.
/anaconda3/envs/spacepost/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## Steps done so far

- Pictures from sprite kit cropped (113 in total)
- Picures resized to 50 X50 pixels

## Steps needed
- Need real/ 3d pictures for mapping


In [3]:
def P2P_U_net_Generator():
    """
    This U_net style CNN will  be used  as the generator for generating images

    https://arxiv.org/pdf/1505.04597.pdf  For the method behind the madness
        
    """

    leaky_relu_alpha = 0.3 # needed for the down sampling
    
    
    upsampling_size = 2
    dropout = 0.25       # drop out of 1/4th of the data to keep the network learning will adjust based on tests later on
    output_channels = 1 # Set to one as each loop in the training will have a single output
                        # channel for the discrimator to test on
    input_shape = (50, 50, 1) # image dimensions for this test set will be set to 50px by 50px
    input_layer = Input(shape=input_shape) # 
    kernel_size = 4
    stride_size = 2

   


    # Encoder Network
    
    # This network is used to down sample the input images
    # Padding is kept at the same since it is fully convoluted

    # 1st Convolutional block in the encoder network
    encoder1 = Convolution2D(filters=64, kernel_size=kernel_size, padding='same',strides=stride_size)(input_layer)
    # no need for batch normalization here! I'ts the first layer
    encoder1 = LeakyReLU(alpha=leaky_relu_alpha)(encoder1)

    # 2nd Convolutional block in the encoder network
    encoder2 = Convolution2D(filters=128, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder1)
    encoder2 = BatchNormalization()(encoder2)

    encoder2 = LeakyReLU(alpha=leaky_relu_alpha)(encoder2)

    # 3rd Convolutional block in the encoder network
    encoder3 = Convolution2D(filters=256, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder2)
    encoder3 = BatchNormalization()(encoder3)
    encoder3 = LeakyReLU(alpha=leaky_relu_alpha)(encoder3)

    # 4th Convolutional block in the encoder network
    encoder4 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder3)
    encoder4 = BatchNormalization()(encoder4)
    encoder4 = LeakyReLU(alpha=leaky_relu_alpha)(encoder4)

    # 5th Convolutional block in the encoder network
    encoder5 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder4)
    encoder5 = BatchNormalization()(encoder5)
    encoder5 = LeakyReLU(alpha=leaky_relu_alpha)(encoder5)

    # 6th Convolutional block in the encoder network
    encoder6 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder5)
    encoder6 = BatchNormalization()(encoder6)
    encoder6 = LeakyReLU(alpha=leaky_relu_alpha)(encoder6)

    # 7th Convolutional block in the encoder network
    encoder7 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder6)
    encoder7 = BatchNormalization()(encoder7)
    encoder7 = LeakyReLU(alpha=leaky_relu_alpha)(encoder7)

    # 8th Convolutional block in the encoder network
    encoder8 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same',strides=stride_size)(encoder7)
    encoder8 = BatchNormalization()(encoder8)
    encoder8 = LeakyReLU(alpha=leaky_relu_alpha)(encoder8)

    # Decoder Network

    # 1st Upsampling Convolutional Block in the decoder network
    decoder1 = UpSampling2D(size=upsampling_size)(encoder8)
    decoder1 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same')(decoder1)
    decoder1 = BatchNormalization()(decoder1)
    decoder1 = Dropout(dropout)(decoder1)
    decoder1 = concatenate([decoder1, encoder7], axis=3)
    decoder1 = Activation('relu')(decoder1)

    # 2nd Upsampling Convolutional block in the decoder network
    decoder2 = UpSampling2D(size=upsampling_size)(decoder1)
    decoder2 = Convolution2D(filters=1024, kernel_size=kernel_size, padding='same')(decoder2)
    decoder2 = BatchNormalization()(decoder2)
    decoder2 = Dropout(dropout)(decoder2)
    decoder2 = concatenate([decoder2, encoder6])
    decoder2 = Activation('relu')(decoder2)

    # 3rd Upsampling Convolutional block in the decoder network
    decoder3 = UpSampling2D(size=upsampling_size)(decoder2)
    decoder3 = Convolution2D(filters=1024, kernel_size=kernel_size, padding='same')(decoder3)
    decoder3 = BatchNormalization()(decoder3)
    decoder3 = Dropout(dropout)(decoder3)
    decoder3 = concatenate([decoder3, encoder5])
    decoder3 = Activation('relu')(decoder3)

    # 4th Upsampling Convolutional block in the decoder network
    decoder4 = UpSampling2D(size=upsampling_size)(decoder3)
    decoder4 = Convolution2D(filters=1024, kernel_size=kernel_size, padding='same')(decoder4)
    decoder4 = BatchNormalization()(decoder4)
    decoder4 = concatenate([decoder4, encoder4])
    decoder4 = Activation('relu')(decoder4)

    # 5th Upsampling Convolutional block in the decoder network
    decoder5 = UpSampling2D(size=upsampling_size)(decoder4)
    decoder5 = Convolution2D(filters=1024, kernel_size=kernel_size, padding='same')(decoder5)
    decoder5 = BatchNormalization()(decoder5)
    decoder5 = concatenate([decoder5, encoder3])
    decoder5 = Activation('relu')(decoder5)

    # 6th Upsampling Convolutional block in the decoder network
    decoder6 = UpSampling2D(size=upsampling_size)(decoder5)
    decoder6 = Convolution2D(filters=512, kernel_size=kernel_size, padding='same')(decoder6)
    decoder6 = BatchNormalization()(decoder6)
    decoder6 = concatenate([decoder6, encoder2])
    decoder6 = Activation('relu')(decoder6)

    # 7th Upsampling Convolutional block in the decoder network
    decoder7 = UpSampling2D(size=upsampling_size)(decoder6)
    decoder7 = Convolution2D(filters=256, kernel_size=kernel_size, padding='same')(decoder7)
    decoder7 = BatchNormalization()(decoder7)
    decoder7 = concatenate([decoder7, encoder1])
    decoder7 = Activation('relu')(decoder7)

    # Last Convolutional layer
    decoder8 = UpSampling2D(size=upsampling_size)(decoder7)
    decoder8 = Convolution2D(filters=output_channels, kernel_size=kernel_size, padding='same')(decoder8)
    decoder8 = Activation('tanh')(decoder8)

    generative_model = Model(inputs=[input_layer], outputs=[decoder8])
    return generative_model


In [28]:


stride_size_descrim = 2
kernel_size = 4
leaky_relu_alpha_descrim = 0.3
num_filters_start = 64  # Number of filters to start with
num_kernels = 100
kernel_dim = 5
discrim_output_shape = (50, 50, 1) # Image shape will follow the same pixels at the U_net generator above
discrim_input_shape = (50, 50, 1) # Image output shape should have the same pixel deminsion as the genrator network

# number of patches for discriminator
numb_of_patches = int((discrim_output_shape[0] / discrim_input_shape[0]) * (discrim_output_shape[1] / discrim_input_shape[1]))


def p2p_discriminator():
    """
    The Architecture of the discrminator is based off of that of the Patch GAN as described by the following paper:
    
    https://arxiv.org/abs/1611.07004
    
    """
    # Define the input layer for the network
    input_layer = Input(shape=discrim_input_shape)
    
    descriminator = Convolution2D(filters=64, kernel_size=kernel_size, padding='same', strides=stride_size)(input_layer)
    descriminator = LeakyReLU(alpha=leaky_relu_alpha_descrim)(descriminator)
    
    # Calculate the number of convolutional layers in this run
    total_convolutional_layers = int(np.floor(np.log(discrim_output_shape[1]) / np.log(2)))
    
    filters_list = [num_filters_start * min(total_convolutional_layers, (2 ** i)) for i in range(total_convolutional_layers)]
    
    # Using a list slice to get the next 7 layers in an orderly fashion
    for filters in filters_list[1:]:
        descriminator = Convolution2D(filters=filters, kernel_size=kernel_size, padding=padding, strides=stride_size_descrim)(descriminator)
        descriminator = BatchNormalization()(descriminator)
        descriminator = LeakyReLU(alpha=leaky_relu_alpha_descrim)(descriminator)
        
        
    # Adding a flatten layer to the network to turn the next step into a 1D tensor for the softmax coming up
    flattener = Flatten()(descriminator)
    
    # Softmax Layer
    dense_layer = Dense(units=2, activation='softmax')(flattener)
    
    # Creates a keras model for the patch gans discriminator network
    model_discriminator = Model(inputs=[input_layer], outputs=[dense_layer, flattener])
    
    # Create a list of input layers equal to number of patches
    input_layers_list = [Input(shape=discrim_input_shape) for _ in range(numb_of_patches)]
    
    # Pass the patches to the Model discriminator and get the probability distribution for the model
    output1 = [model_discriminator(patch)[0] for patch in input_layers_list]
    output2 = [model_discriminator(patch)[1] for patch in input_layers_list]
    
    # In case of multiple patches, concatinate to calculate loss
    if len(output1) > 1:
        output1 = concatenate(output1)
    else:
        output1 = output1[0]

    # if multiple patches exist, merge output2 as well
    if len(output2) > 1:
        output2 = concatenate(output2)
    else:
        output2 = output2[0]
        
    # Add a dense layer
    dense_layer2 = Dense(num_kernels * kernel_dim, use_bias=False, activation=None)

    # Add a lambda layer taken from a textbook that works fine with this network untill the return
    custom_loss_layer = Lambda(lambda x: K.sum(
            K.exp(-K.sum(K.abs(K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, pattern=(1, 2, 0)), 0)), 2)), 2))

    # Pass the output2 tensor through dense_layer2
    output2 = dense_layer2(output2)

    # Reshape the output2 tensor to fit the kernel dimnetions
    output2 = Reshape((num_kernels, kernel_dim))(output2)

    # Pass the output2 tensor through the custom_loss_layer
    output2 = custom_loss_layer(output2)

    # Finally concatenate output1 and output2
    output1 = concatenate([output1, output2])
    
    # pass ot to a sfotmax function to get probablities
    final_output = Dense(2, activation="softmax")(output1)

    # Create a discriminator model
    discriminator = Model(inputs=input_layers_list, outputs=[final_output])
    
    # Return discriminator
    return discriminator
    


In [29]:
p2p_discriminator()